In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [294]:
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from nltk.tokenize import sent_tokenize

In [300]:
import numpy as np

In [283]:
# Extracting Articles from URLs and saving into .txt file

for i in range(0,170):
    content = ""
    file_name = str(i+1)+".txt"
    url = input['URL'][i]
    page = requests.get(url,headers={"User-Agent": "XY"})    
    soup = BeautifulSoup(page.content,'html.parser')    
    title = soup.findAll('h1',attrs = {'class':'entry-title'})
    title = title[0].text
    article = soup.findAll(attrs = {'class':'td-post-content'})
    text_article = [t for t in article[0].select(':not(pre)')]
    content = ""
    for tag in text_article:
        content = content + tag.text
    content = content.replace(u'\xa0', u' ')
    with open(file_name,'w',encoding="utf-8") as f:
        f.write(content)
        f.close()

In [297]:
#All Methods to get values of attributes

def sentiment_analysis(text,cleaned_text):
    score = SentimentIntensityAnalyzer().polarity_scores(text)
    polarity_score = (score['pos'] - score['neg'])/((score['pos']+score['neg'])+0.000001)
    subjectivity_score = (score['pos'] + score['neg'])/(len(cleaned_text)+0.000001)
    return [score['pos'],score['neg'],polarity_score,subjectivity_score]

def readability(word_count,sentences,complex_words):
    avg_sen_len = word_count/sentences
    percent_Complex_Words = complex_words/word_count
    fog_index = 0.4*(avg_sen_len+percent_Complex_Words)
    words_per_sen = word_count/sentences
    return [avg_sen_len,percent_Complex_Words,fog_index,words_per_sen]

#Syllable Count

def sylco(word) :
    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

#complex words count

def complex_word_count(word_list):
    complex_words = len(list(filter(lambda word: sylco(word) > 2,word_list)))
    return complex_words

#finding pronouns

def pronouns(text):
    pattern = re.compile(r'I|me|mine|we|my|ours|us|our')
    count = 0
    matches = pattern.finditer(text)
    for match in matches:
        count = count+1
    return count

#average word length

def avgWordLen(cleaned_text):
    words = len(cleaned_text)
    chars = [len(word) for word in cleaned_text]
    chars = sum(chars)
    return chars/words

In [303]:
#Stop Words

stop_words = open("StopWords.txt",encoding = "utf-8").read()
stop_words = stop_words.split('\n')

output = pd.read_csv('output.csv')
cols = ['POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH',
        'PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT',
        'SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']
for i in range(0,170):
    final_values = []
    file_name = str(i+1)+".txt"
    text = open(file_name,encoding = "utf-8").read()
    text_lower = text.lower()
    sentences = len(sent_tokenize(text_lower))
    #cleaning punctuations and stop words
    
    punctuation_free = text_lower.translate(str.maketrans('','',string.punctuation))
    punctuation_free_list = punctuation_free.split()
    cleaned_text = []

    #removing stop words

    for word in punctuation_free_list:
        if word not in stop_words:
            cleaned_text.append(word)
            
    #positive score, negative score, polarity score, subjectivity score
    
    sentiment = sentiment_analysis(text_lower,cleaned_text)
    complex_words = complex_word_count(cleaned_text)
    word_count = len(cleaned_text)
    read = readability(word_count,sentences,complex_words)
    syllables = sum([sylco(word) for word in cleaned_text])/word_count
    personal_pronouns = pronouns(text_lower)
    avg_word_len = avgWordLen(cleaned_text)
    final_values = [sentiment,read,complex_words,word_count,syllables,personal_pronouns,avg_word_len]
    final_values = list(np.hstack(final_values))
    output.loc[i,cols] = final_values

In [307]:
output.to_csv('final_output.csv',index = False)